In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix, ConvexHull

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        np_XYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(np_XYs[:, 0]):
            XY = np_XYs[np_XYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def plot(path_XYs):
    colours = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    for i, XYs in enumerate(path_XYs):
        c = colours[i % len(colours)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()


In [2]:
def combine_polylines(polylines, threshold=10):
    points = np.vstack([point for polyline in polylines for point in polyline])
    dist_matrix = distance_matrix(points, points)
    
    clusters = []
    visited = set()
    
    for i, polyline in enumerate(polylines):
        if i in visited:
            continue
        cluster = [i]
        queue = [i]
        visited.add(i)
        
        while queue:
            idx = queue.pop(0)
            neighbors = np.where(dist_matrix[idx] < threshold)[0]
            for neighbor in neighbors:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    cluster.append(neighbor)
        
        clusters.append(cluster)
    
    combined_shapes = [np.vstack([polylines[i] for i in cluster]) for cluster in clusters]
    return combined_shapes


In [3]:
def extract_features_from_shape(points):
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    
    num_vertices = len(hull_points)
    perimeter = np.sum(np.linalg.norm(np.diff(hull_points, axis=0), axis=1))
    area = hull.volume
    compactness = perimeter ** 2 / area
    angles = []
    for i in range(num_vertices):
        a = hull_points[i]
        b = hull_points[(i + 1) % num_vertices]
        c = hull_points[(i + 2) % num_vertices]
        ba = a - b
        bc = c - b
        angle = np.arccos(np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc)))
        angles.append(angle)
    
    features = {
        'num_vertices': num_vertices,
        'perimeter': perimeter,
        'area': area,
        'compactness': compactness,
        'angles': angles
    }
    
    return features


In [4]:
def classify_shape(features):
    num_vertices = features['num_vertices']
    angles = features['angles']
    compactness = features['compactness']
    
    if num_vertices == 2:
        return 'Straight Line'
    elif np.allclose(angles, np.pi / 2, atol=0.1):
        if compactness > 14:
            return 'Rounded Rectangle'
        else:
            return 'Rectangle'
    elif num_vertices > 4 and np.std(angles) < 0.1:
        return 'Regular Polygon'
    elif num_vertices > 4 and np.std(angles) > 0.1:
        return 'Star Shape'
    elif num_vertices == 1 and compactness < 12:
        return 'Circle'
    elif num_vertices == 1 and compactness >= 12:
        return 'Ellipse'
    else:
        return 'Unknown Shape'

def regularize_shape(shape, points):
    if shape == 'Straight Line':
        return regularize_line(points)
    elif shape == 'Circle':
        return regularize_circle(points)
    elif shape == 'Ellipse':
        return regularize_ellipse(points)
    elif shape == 'Rectangle':
        return regularize_rectangle(points)
    elif shape == 'Rounded Rectangle':
        return regularize_rounded_rectangle(points)
    elif shape == 'Regular Polygon':
        return regularize_polygon(points, num_sides=6)  # Example for hexagon
    elif shape == 'Star Shape':
        return regularize_star(points, num_points=5)  # Example for star with 5 points
    else:
        return points

def regularize_line(points):
    return points  # Simplified example

def regularize_circle(points):
    return points  # Simplified example

def regularize_ellipse(points):
    return points  # Simplified example

def regularize_rectangle(points):
    return points  # Simplified example

def regularize_rounded_rectangle(points):
    return points  # Simplified example

def regularize_polygon(points, num_sides=6):
    return points  # Simplified example

def regularize_star(points, num_points=5):
    return points  # Simplified example


In [5]:
def detect_symmetry(points):
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    center = np.mean(hull_points, axis=0)
    symmetric = True
    
    for i in range(len(hull_points) // 2):
        if not np.allclose(hull_points[i] - center, center - hull_points[-(i+1)], atol=1e-1):
            symmetric = False
            break
    
    return symmetric

def enhance_symmetry(points):
    if detect_symmetry(points):
        return points
    else:
        # Apply symmetry enhancement (e.g., averaging mirrored points)
        hull = ConvexHull(points)
        hull_points = points[hull.vertices]
        center = np.mean(hull_points, axis=0)
        new_points = []

        for i in range(len(hull_points)):
            mirrored_point = 2 * center - hull_points[i]
            new_points.append((hull_points[i] + mirrored_point) / 2)

        return np.array(new_points)


In [6]:
def complete_shapes(closed_curves):
    completed_curves = []
    
    for curve in closed_curves:
        completed_curve = curve.copy()
        
        # Find neighboring curves and connect them
        for neighbor in closed_curves:
            if curve is neighbor:
                continue
            
            # Check for shared boundary
            shared_points = set(tuple(p) for p in curve).intersection(set(tuple(p) for p in neighbor))
            
            if shared_points:
                completed_curve = np.vstack([completed_curve, neighbor])
        
        completed_curves.append(completed_curve)
    
    return completed_curves


In [7]:
def process_fragments(path_XYs):
    combined_shapes = []
    for shape in path_XYs:
        combined_shape = combine_polylines(shape)
        combined_shapes.extend(combined_shape)

    features_list = [extract_features_from_shape(shape) for shape in combined_shapes]

    regularized_shapes = []
    for shape, features in zip(combined_shapes, features_list):
        shape_type = classify_shape(features)
        regularized_shape = regularize_shape(shape_type, shape)
        regularized_shapes.append(regularized_shape)

    symmetric_shapes = [enhance_symmetry(shape) for shape in regularized_shapes]

    completed_shapes = complete_shapes(symmetric_shapes)

    return completed_shapes

# Example usage
csv_path = 'frag0.csv'  # Update with your CSV file path
path_XYs = read_csv(csv_path)
processed_shapes = process_fragments(path_XYs)

# Plot the processed shapes
plot(processed_shapes)


FileNotFoundError: frag0.csv not found.